In [ ]:
# -*- coding: utf-8 -*-

#  radiship.py
#
import cv2
import matplotlib
import matplotlib.pyplot as plt
from thin.thin import getSkelline
import numpy as np
import random,copy

''' ------------------------------------
   cv2 用の画像を matplotlib の imshow で表示するための関数
'''

# 表示をオンオフするためのフラグ
IMGON = True

# プロット用関数
def plotimg(img,layout="111"):
    if IMGON is False: return
    if len(img.shape) == 2:
        pltgry(img,layout)
    elif len(img.shape) ==3:
        pltcol(img,layout)

def pltgry(img,layout="111"):
    plt.subplot(layout)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_GRAY2RGB))

def pltcol(img,layout="111"):
    plt.subplot(layout)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    
''' ------------------------------------
   ２階調化と膨張収縮によるゴミ除去
   
   ２階調化した後 
   MORPH_OPEN（収縮後膨張） を２回実行して孤立点や線幅１の髭を除去×２回
   MORPH_CLOSE（膨張後収縮） を２回実行して穴埋め×２回
'''
def dthreshold(image, thres=70, open=True, close=True):
    ret,bw=cv2.threshold(image,thres,255,cv2.THRESH_BINARY)
    kernel = np.ones((5,5),np.uint8)
    if open: bw=cv2.morphologyEx(bw,cv2.MORPH_OPEN, kernel,iterations = 2)
    if close: bw=cv2.morphologyEx(bw,cv2.MORPH_CLOSE, kernel,iterations = 2)
    return bw

''' ------------------------------------
    ラベルイメージから小領域を削除する
'''
def removeSmallArea(numberoflabels, labelimage, stats, threshold):
    img= np.zeros(labelimage.shape,dtype=np.uint8)
    for i in range(1,numberoflabels ):
        if  stats[i] > threshold:
            img = img + np.uint8(labelimage==i)*255
    return img   

def getBigWhite(image):
    (numberoflabels, labelimage, menseki) = labeling(image,connectivity=4,b_or_w=1)
    # 1番大きな領域がダイコンの主要部分であると仮定する。
    dnum = np.argmax(menseki[1:]) #1番以降を取りだしたときに一番大きい要素を持つ index 
    return np.uint8(labelimage==dnum+1)*255
    
''' ------------------------------------
ラベリングして色つけ
    image  2値画像
    connectivity  8連結型か4連結型かのスイッチ(default 8 )
        4連結の場合は斜めに隣接していても連続していないと考える
    bw 1 なら白領域のラベリング、０なら黒領域のラベリング
    
        関数　
            labeling()  ->  ラベリングしてラベル数、ラベル画像、情報を返す  (gray)
            color2label()  -> ラベル画像に色づけして返す（color)
            colorlabeling()  ->  ラベリングして色付けしてカラー画像を返す（color)
        
    返り値  (numberoflabels, labelimg, menseki) 
         numberoflabels  ラベルの数（反対色全体も０のラベルがつくので１多くなる）  
         labelimg　各画素の明るさデータの代わりにラベルデータが入った画像
         menseki 各ラベルのつけられた画素数のリスト
'''
# image に連結成分ごとのラベルをつけて画像として返す関数
def labeling(image,connectivity=8,b_or_w=1):
    img = np.array(image,dtype=np.uint8)
    if b_or_w == 0: img = ~img
    con = cv2.connectedComponentsWithStats(img, connectivity)
    numberoflabels = con[0]
    labelimg = con[1]
    menseki  = con[2][:,cv2.CC_STAT_AREA]
    return (numberoflabels, labelimg, menseki) 

# ラベル画像に色づけして返す関数
def color2label(labelimage, numberoflabels):
    colors = [ [0,0,0],  # black
                    [255,255,255], # white
                    [255,255,0], # yellow
                    [0,255,255], # aqua
                    [0,255,0], # lime
                    [128,128,0], # olive
                    [255,182,193], # light pink
                    [175,238,238], # paleturquoise
                    [173,255,47], # greeyellow
                    [255,215,0]] # gold
    for  i in range(10,numberoflabels+2):
            colors.append(np.array([random.randint(100, 255), random.randint(0, 255), random.randint(0, 255)]))
    print(colors)
    s = labelimage.shape
    colorimage =np.zeros((s[0],s[1],3), dtype=np.uint8)
    
    for y in range(0, s[0]):
       for x in range(0, s[1]):
           if labelimage[y][x] > 0:
                colorimage[y][ x] = colors[labelimage[y][x]]
           else:
                colorimage[y][x] = [0, 0, 0]
    return colorimage

# 上の２つを一気に実行する関数
def colorlabeling(image,connectivity=8,bw=1):
    (numberoflabels, labelimg, menseki) = labeling(image,connectivity,bw)
    colorimage = color2label(labelimg, numberoflabels)
    return (numberoflabels, colorimage, menseki) 

''' ------------------------------------
線図形のセグメンテーション
'''
# 指定されたピクセルの周囲のピクセルを取得
def neighbours(x, y, image):
    ns = [image[y][x+1],image[y-1][x+1],image[y-1][x], image[y-1][x-1],image[y][x-1],  # X１, X２, X３, X４, X5
    image[y+1][x-1],image[y+1][x], image[y+1][x+1],image[y][x+1], image[y-1][x+1] ]    # X6, X7, X8,X1,X2 
    for i in range(10): ns[i] = ns[i]/255
    return ns

# 連結数 (8連結)
def connection_number(neighbours):
    n = 1-np.array(neighbours)
    return sum((1-n)[:-2]), sum( n[i] -  n[i]*n[i+1]*n[i+2]  for  i in range(0,7,2)) 

# スケルトンのリストを生成する関数、細線化バージョン
def skellist(skel, dist):
    # skel 細線化済み画像データ 完全に８連結幅１のひとつながりの線画であることが前提
    # dist 距離画像データ
    # bwの白(255)連結成分を細線化し、残った画素の座標と距離データからなるタプルのリストを返す
    (height,width) = skel.shape
    result = []
 
    for i in range(1,height-1):
        for j in range(1,width-1):
            if skel[i][j]>0: 
                '''
                numberof1, cn = connection_number(neighbours(j,i,skel))
                # この時点でまだ消去可能な画素が残っていれば削除する
                if  skel[i][j] == 255 and  cn==1 and numberof1 > 1  : 
                    skel[i][j] = 0
                else:
                    result.append([i,j,dist[i][j]])'''
                result.append([i,j,dist[i][j]])
    
    return result

''' ------------------------------------
スケルトンからの２値画像データの復元
'''
# スケルトンデータから2値画像データを復元 （検証のため）
def skel2img(skeldata,shape,skelimg):
    img = np.zeros(shape,dtype=np.uint8)
    zeroimg = np.zeros(skelimg.shape,dtype=np.uint8)
    mask = cv2.merge((skelimg,skelimg,zeroimg))
    for i in range (len(skeldata)):
        cv2.circle(img,(skeldata[i][1],skeldata[i][0]),int(skeldata[i][2]),(256,256,256),-1)
    plt.imshow(mask)
    return cv2.bitwise_xor(img,mask)  

'''' ------------------------------------
スケルトンを分岐点で分割

完全に幅１の８連結線画であることが前提
'''
def skllabel(skelimg,  skeldata):
    con = True
    tmpimg = np.array(skelimg,dtype=np.uint8)
    count = 0
    while con:
        con = False
        for skelton in skeldata:
            x = skelton[1]
            y = skelton[0]
            numberof1, cn = connection_number(neighbours(x,y,tmpimg))
            if tmpimg[y][x] == 255 and cn > 2 :
                # 線画から分岐点の画素を除去する
                con = True
                count += 1
                tmpimg[y][x] = 0
    if count == 0:
        out = (2,skelimg,[-1,-1])
    else:
        out = labeling(tmpimg,connectivity=8,b_or_w=1)
    return  out #(ラベル数，　ラベル画像，　ラベル諸表のリストが返り値）

''' ------------------------------------
スケルトンの髭を除去する
'''
def oneSkelton(skelimg, skeldata):
    # まず分岐点で線画を分割してラベル画像を得る
    (ns, labelimage, specs) = skllabel(skelimg,  skeldata)
    if ns == 2:   # ラベル数が２のときは何もしなくていい
        pass
    else:
        con = True
        while con:
            tmpimg = np.array(skelimg,dtype=np.uint8)  # 元のイメージをコピー
            shortestsegment = np.argmin(specs)  # 一番短いセグメント番号
            for y in range(0,skelimg.shape[0]):
                for x in range(0,skelimg.shape[1]):
                    if labelimage[y][x] == shortestsegment:
                        tmpimg[y][x] = 0  # 画素を削除する
            check = labeling(tmpimg,connectivity=8,b_or_w=1)
            numberofregions = check[0]
            if numberofregions > 2:  # もともと２なのに増えたということは中間のセグメントであった
                specs = specs[:shortestsegment]+specs[shortestsegment+1:]
            else:
                skelimg = tmpimg
                sknew = []
                for d in skeldata:
                    if  skelimg[d[0]][d[1]] > 0 :
                        sknew.append(d)
                skeldata = sknew
                (ns, labelimage, specs) = skllabel(skelimg,  skeldata) 
                if ns == 2:
                    con = False
    return skelimg, skeldata

# スケルトンの位置データ補正
def recalcDistanceP(skdata,normP):
    skdpbackup=np.array(skdata)
    # 曲線に沿った距離を求める。理屈では積分していけばいいが
    # デジタル画像では近い画素間の距離は誤差が大きいので10点ごとに
    # 基準点を設け、基準点間の距離の積算＋最寄りの基準点からの距離を
    # 曲線に沿った距離の近似に使う
    xnorm = skdata[normP][1]
    ynorm = skdata[normP][0]
    cnt = 0 # ベースからの画素数 
    accdistAtBase = 0 # 基準点から現在のベースまでの距離の積算
    predist = skdpbackup[0][3]  # 先端のデータ（最も誤差が大きいはず）
    for sk in skdata[normP-1::-1]:
            y = sk[0]
            x = sk[1]
            sk[3] = accdistAtBase - np.sqrt((x-xnorm)**2 + (y - ynorm)**2) 
            cnt = cnt+1
            if cnt==10 :
                accdistAtBase = sk[3]
                (ynorm,xnorm) = (y,x)
                cnt = 0
    print(u"左側修正量 {} ({} -> {})".format(predist-skdata[0][3],predist,skdata[0][3]))
    xnorm = skdata[normP][1]
    ynorm = skdata[normP][0]
    cnt = 0 # ベースからの画素数 
    accdistAtBase = 0 # 基準点から現在のベースまでの距離の積算
    predist = skdpbackup[-1][3]  # 葉元のデータ（こちらも誤差が大きいはず）
    for sk in skdata[normP+1::]:
            y = sk[0]
            x = sk[1]
            sk[3] = accdistAtBase + np.sqrt((x-xnorm)**2 + (y - ynorm)**2) 
            cnt = cnt+1
            if cnt==10 :
                accdistAtBase = sk[3]
                (ynorm,xnorm) = (y,x)
                cnt = 0
    print(u"右側修正量 {} ({} -> {})".format(skdata[-1][3]-predist,predist,skdata[-1][3]))


'''
描画関数

radiusfunc(invert)
軸に沿った距離を横軸，　その点の距離データを縦軸にしたグラフを描く
（距離データとは軸上の点から表皮までの最短距離．　　中央辺りでは径に相当する

'''
def radiusfunc(invert=True):
    if invert:
        for d in skdp: d[3]=-d[3]
    skdN = skdp[skdp[:,3]<= 0] # 基準点より左のデータ
    skdP = skdp[skdp[:,3] > 0] # 基準点より右のデータ
    
    # 横軸を基点からのスケルトン位置までの距離，縦軸をそのスケルトンの距離データとしたグラフ                          
    plt.axis('equal')    
    plt.hold(True)
    ydata = np.array(skdN[:,2])
    xdata = np.array(skdN[:,3])
    plt.plot(xdata,ydata,'-',color=(1,0,0.0))
    plt.plot(xdata,-ydata,'-',color=(1,0,0.0))
    ydata = np.array(skdP[:,2])
    xdata = np.array(skdP[:,3])
    plt.plot(xdata,ydata,'-',color=(0,0,1.0))
    plt.plot(xdata,-ydata,'-',color=(0,0,1.0))
    plt.hold(False)
    plt.show()
    
    
# スケルトンを直線状に再配置し、曲がりを補正した画像を生成する
def makeNormalizedImage(skdp):
    print("中心軸の画素数={}".format(len(skdp)))
    maxh = np.max(skdp[:,2]) # もっとも近い輪郭までの距離
    maxw = np.max(skdp[:,3]) # 基準点からの距離の最大値
    minw = np.min(skdp[:,3]) # 基準点からの距離の最小値
    nimg = np.zeros((int(3*maxh),int(1.5*(maxw-minw)),3),dtype=np.uint8) # 50%マージンで描画用エリアを確保
    cent = np.array((int(1.5*(-minw)),int(1.5*maxh))) # 基準点の画像内座標
    for i in range(len(skdp)):
        center = (int(skdp[i][3])+cent[0],cent[1])
        radius = int(skdp[i][2])
        color = (255,255,255) # White
        thickness = -1 # 塗りつぶし
        nimg = cv2.circle(nimg,center,radius,color,thickness)
    mask = nimg[:,cent[0]:,:]
    cv2.threshold(mask,127,128,cv2.THRESH_BINARY,dst=mask)
    nimg = cv2.cvtColor(nimg,cv2.COLOR_BGR2GRAY)
    return nimg

# 輪郭線の抽出
# mode 0  全体の輪郭　　mode 1 最大径より先だけの輪郭  mode 2 先の上半分だけ
def  getContour(img, mode):
    center = img.shape[0]/2
    if mode == 0:
        ret, bw = cv2.threshold(img,0,255,cv2.THRESH_BINARY)        
    else:
        ret, bw = cv2.threshold(img,200,255,cv2.THRESH_BINARY)  
    image, contours, hierarchy = cv2.findContours(bw,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    if  mode ==2:
        cnt = []
        maxx = np.max(contours[:][0][1])
        for i in contours[0]:
            if i[0][1] >= center and i[0][0]<maxx :
                cnt.append([list(i[0])])
            contours = np.array([cnt])
    return image,contours


# 先端部分だけの点列の生成
def getTipData(image):
    bw,cnt = getContour(image,mode=2)
    xdata = [i[0][0]  for i in cnt[0]]
    ydata = [i[0][1]- bw.shape[0]/2 for i in cnt[0]] 
    return xdata,ydata

# yの値が terget となる位置が x=0 となるようにデータ全体を x 方向にシフトさせる
def shiftX(dx,dy,terget):
         found = False
         offset = 0.0
         for (x,y) in zip(dx,dy):
            if not found and y >terget:
                found = True
                offset = x
         dx = dx - offset
         return dx

#  maxR を基準としてサイズを正規化 
def sizeNormalize(xdata,ydata, norm, target):
    xdataR = np.array(xdata)/norm 
    ydataR= np.array(ydata)/norm
    xdataR= shiftX(xdataR,ydataR,target)    
    return xdataR, ydataR
